In [1]:
import re
import sys
sys.path.append(r"D:\往期\QHJ\ZERO")
sys.path.append(r"E:\dataparse\Python_DATA_PARSE\QHJ\ZERO")
import warnings
warnings.filterwarnings("ignore")

from pyquery import PyQuery as pq
import requests
from urllib.parse import urlencode,parse_qs
import numpy as np
import pandas as pd
import redis

import pymysql

import json
from tools import *

qm = QHJMall(MALL_KEY)


import datetime

sku_goods = pd.read_excel(r'D:\奇货居\素材\商城图片素材\商品信息.xlsx',sheet_name='商品详情') \
                [['商品ID', '发货商','发货商ID','goods_type','规格','单位', '市场价','售价','商品名简称']] \
                .rename(columns={"售价":"单价","商品名简称":"商品名"}).fillna({"goods_type":"original"})

free_goods = sku_goods[sku_goods['goods_type']=='free']


In [ ]:
clear_folder(1,is_recursion=False)

In [31]:
l = []
for order in qm.gen_orders("free",order_status=2):
    l.append(order)
    
data = pd.DataFrame(l)


# 过滤重复项
data.drop_duplicates('订单号',inplace=True)
# conn = redis.Redis(**REDIS_MALL_ORDER_DIC)
# def dup_order(order):
#     if conn.sismember('order:old',order):
#         return None
#     return order

# data['订单号'] = data['订单号'].apply(dup_order)
data.dropna(subset=['订单号'],inplace=True)

data = data.replace("三金美肤面膜 缓解过敏 修复护理|三金美肤面膜 维稳修护 强化屏障", "三金护肤面膜 维稳修护 强化屏障", regex=True)
patt = '三金补水面膜 镇静维稳 深层补水\\(补水\\)|三金美肤面膜 美白功效 健康肤色\\(美肤\\)|三金护肤面膜 维稳修护 强化屏障\\(护肤\\)|三金水光针\\(水光针\\)'
data['商品名'] = data['订单详情'].str.extract('(?P<商品名>%s)' % patt, expand=False).str.strip()


data = pd.concat([data['用户信息'].str.extract("""
        (?<=下单人：)(?P<收件人>[^\n]+)
        \s+手机：(?P<联系方式>[^\n]+)
        \s+收货地址：(?P<收货地址>[^\n]+)""", flags=re.S | re.X),data],axis=1)

data = pd.merge(data
                             , free_goods, on='商品名', how='left')

data['支付金额'] = 0
data.reset_index(drop=True, inplace=True)

请求 {'order_status': 2, 'order_time': '', 'order_id': '', 'order_type': 'free', 'page': 1}
请求 {'order_status': 2, 'order_time': '', 'order_id': '', 'order_type': 'free', 'page': 15}
请求 {'order_status': 2, 'order_time': '', 'order_id': '', 'order_type': 'free', 'page': 14}
请求 {'order_status': 2, 'order_time': '', 'order_id': '', 'order_type': 'free', 'page': 13}
请求 {'order_status': 2, 'order_time': '', 'order_id': '', 'order_type': 'free', 'page': 12}
请求 {'order_status': 2, 'order_time': '', 'order_id': '', 'order_type': 'free', 'page': 11}
请求 {'order_status': 2, 'order_time': '', 'order_id': '', 'order_type': 'free', 'page': 10}
请求 {'order_status': 2, 'order_time': '', 'order_id': '', 'order_type': 'free', 'page': 9}
请求 {'order_status': 2, 'order_time': '', 'order_id': '', 'order_type': 'free', 'page': 8}
请求 {'order_status': 2, 'order_time': '', 'order_id': '', 'order_type': 'free', 'page': 7}
请求 {'order_status': 2, 'order_time': '', 'order_id': '', 'order_type': 'free', 'page': 6}
请求 {

In [40]:
save_dir = r"D:\Downloads\QHJ_MALL"
save = SaveXl(save_dir)
data['导出订单时间'] = save.suf.strip()

b


fields = ["订单号","发货商",'商品ID',"数量","支付金额","备注","收件人","联系方式","收货地址","goods_type",
          "导出订单时间"  ,'下单时间', '支付时间']


data =  data[fields]
sql = 'insert into order_details(%s) values(%s)' %(','.join(fields),",".join(np.repeat('%s',len(fields))))

# date_time_len = 3
# sql = 'insert into order_details(%s) values(%s,%s)' %(','.join(fields),",".join(np.repeat('%s',len(fields)-date_time_len)),
#         ",".join(np.repeat("str_to_date(%s,'%%Y-%%m-%%d %%H:%%i:%%S')",date_time_len)))

data = data.fillna(value={"备注":""})
dup_order = []
for index,row in data.iterrows():
    
    try:
        
        cursor.execute(sql,row.tolist())
        print(row.tolist())
    except:
        dup_order.append(index)
conn.commit()
conn.close()

In [4]:
stop

NameError: name 'stop' is not defined

In [68]:
# .strftime("%Y-%m-%d")



'2019-11-17'

In [79]:


r_conn = redis.Redis(**REDIS_MALL_ORDER_DIC)
conn = pymysql.connect(**MYSQL_MALL_DIC)
cursor = conn.cursor()
result = cursor.execute("select CONCAT(订单号,'-',商品ID) as k from order_details where 下单时间>='%s';" 
                        %(datetime.date.today() - datetime.timedelta(1)).strftime("%Y-%m-%d"))


r_conn.delete('order:old')
r_conn.sadd('order:old',*[ key[0] for key in cursor.fetchall()])


4

In [ ]:
len(u_data)

In [ ]:
len(data)

In [ ]:
u_data = data

In [ ]:
l = []
f_dir = r"D:\奇货居\work\外发订单\新订单\已发"
for file in os.listdir(f_dir):
    if file.startswith('三金'):
        l.append(pd.read_excel(os.path.join(f_dir,file),index=False,converters={"订单号":str}))
data = pd.concat(l)
data['订单号'] = data['订单号'].astype(str)
data['订单号'][data['订单号'].isin({'149891054111246', '154427419014946', '167197513254485', '501250902978832'})] = \
"0"  + data['订单号'][data['订单号'].isin({'149891054111246', '154427419014946', '167197513254485', '501250902978832'})]

In [ ]:
data.columns

In [ ]:
u_data['订单号'].dtype

In [ ]:
data['订单号'] = data['订单号'].astype(str)

In [ ]:
set(u_data['订单号']) - set(data['订单号']) 

In [28]:
l = []
f_dir = r"D:\奇货居\work\外发订单\新订单\已发\原始数据"
for file in os.listdir(f_dir):
    if file.startswith('订单'):
        
        temp = pd.read_excel(os.path.join(f_dir,file),index=False,converters={"订单号":str})
        temp['导出订单时间'] = file[3:22]
        l.append(temp)
data = pd.concat(l)
data['订单号'] = data['订单号'].astype(str)
data['订单号'][data['订单号'].isin({'149891054111246', '154427419014946', '167197513254485', '501250902978832'})] = \
"0"  + data['订单号'][data['订单号'].isin({'149891054111246', '154427419014946', '167197513254485', '501250902978832'})]
data.rename(columns={'快递方式':"快递公司"},inplace=True)

In [29]:
fileds = ['导出订单时间','订单号']
data_r = data[fileds]
data_r.replace({"快递公司":{r'(.+?)快递$':r"\1"}},regex=True,inplace=True)
conn = pymysql.connect(**MYSQL_MALL_DIC)
cursor = conn.cursor()
sql = "update order_details set 导出订单时间=%s where 订单号=%s"
for index, row in data_r.iterrows():
    print(row.tolist())
    cursor.execute(sql,row.tolist())
#     conn.commit()
conn.commit()
conn.close()

['2019-12-12 15_37_45', '7801733718286446']
['2019-12-12 15_37_45', '19121251101531']
['2019-12-12 15_37_45', '19121248494810']
['2019-12-12 15_37_45', '19121253985148']
['2019-12-12 15_37_45', '19121249485597']
['2019-12-12 15_37_45', '19121252575210']
['2019-12-12 15_37_45', '19121257485710']
['2019-12-12 15_37_45', '19121298101101']
['2019-12-12 15_37_45', '19121210253569']
['2019-12-12 15_37_45', '19121297579856']
['2019-12-12 15_37_45', '19121298574910']
['2019-12-12 15_37_45', '19121297101535']
['2019-12-12 15_37_45', '19121249971025']
['2019-12-12 15_37_45', '19121250575757']
['2019-12-12 15_37_45', '19121248101504']
['2019-12-12 15_37_45', '19121255999954']
['2019-12-12 15_37_45', '19121210210051']
['2019-12-12 15_37_45', '19121251101505']
['2019-12-12 15_37_45', '19121210052989']
['2019-12-12 15_37_45', '19121250985210']
['2019-12-12 15_37_45', '19121299971024']
['2019-12-12 15_37_45', '19121210210010']
['2019-12-13 15_38_09', '5609222359417374']
['2019-12-13 15_38_09', '97844

In [5]:

# 更新订单时间
l = []
f_dir = r"D:\奇货居\work\外发订单\新订单\已发\反馈数据"
for file in os.listdir(f_dir):
    if file.startswith('趣领'):
        l.append(pd.read_excel(os.path.join(f_dir,file),index=False,converters={"订单号":str}))
data = pd.concat(l)
data['订单号'] = data['订单号'].astype(str)
data['订单号'][data['订单号'].isin({'149891054111246', '154427419014946', '167197513254485', '501250902978832'})] = \
"0"  + data['订单号'][data['订单号'].isin({'149891054111246', '154427419014946', '167197513254485', '501250902978832'})]
data.rename(columns={'快递方式':"快递公司"},inplace=True)

In [ ]:
data[data_r['运单号'].isin(set(data[data_r['运单号'].duplicated()]['运单号'].unique()))]

In [19]:
fileds = ['运单号','快递公司','订单号']
data_r = data[fileds]
data_r.replace({"快递公司":{r'(.+?)快递$':r"\1"}},regex=True,inplace=True)
conn = pymysql.connect(**MYSQL_MALL_DIC)
cursor = conn.cursor()
sql = "update order_details set 运单号=%s,快递公司=%s where 订单号=%s"
for index, row in data_r.iterrows():
    print(row.tolist())
    cursor.execute(sql,row.tolist())
#     conn.commit()
conn.commit()
conn.close()

[75319805628524, '中通', '7801733718286446']
[75319805628565, '中通', '19121251101531']
[75319805773360, '中通', '19121248494810']
[75319805628616, '中通', '19121253985148']
[75319805628537, '中通', '19121249485597']
[75319805628578, '中通', '19121252575210']
[75319805923946, '中通', '19121257485710']
[75319805773344, '中通', '19121298101101']
[75319805923933, '中通', '19121210253569']
[75319805628552, '中通', '19121297579856']
[75319805628593, '中通', '19121298574910']
[75319805923959, '中通', '19121297101535']
[75319805773316, '中通', '19121249971025']
[75319805773372, '中通', '19121250575757']
[75319805923961, '中通', '19121248101504']
[75319805773329, '中通', '19121255999954']
[75319805628540, '中通', '19121210210051']
[75319805773316, '中通', '19121251101505']
[75319805628586, '中通', '19121210052989']
[75319805773357, '中通', '19121250985210']
[75319805773331, '中通', '19121299971024']
[75319805628603, '中通', '19121210210010']
[75321112453546, '中通', '2259820089497103']
[75321112453625, '中通', '8691432545841121']
[753211128